In [8]:
import pandas as pd
import numpy as np
import random
import pickle
from gams import *
import os
import sys
import math

cs = 4                         #number of candidate charging station

trip_information = pd.read_csv("Trip_information_NumericalStudy.csv")
terminal_distance = pd.read_csv("Between_terminal_distance_NumericalStudy.csv")
##terminals
#1---University Medical Center
#2---Salt Lake Central Station
#3---This is the Place State Park
#4---North Temple Station

From_depot_to_trip_time = pd.read_csv("From_depot_to_trip_time")
From_trip_to_depot_time = pd.read_csv("From_trip_to_depot_time")
From_depot_to_trip_energy = pd.read_csv("From_depot_to_trip_energy")
From_trip_to_depot_energy = pd.read_csv("From_trip_to_depot_energy")

available_charging_time = pd.read_csv("Between_trip_time_NumericalStudy.csv")
energy_need = pd.read_csv("Between_trip_energy_NumericalStudy.csv")

following_set = pickle.load(open('Following_set_dict.pickle','rb'))

######define a function to get a following trip of a service trip
def return_following(i,following_set):
    following_set_i = following_set[i]
    if len(following_set_i) == 0:
        return -1
    else:
        following_set_i = following_set_i[0:3]
        return(random.choice(following_set_i))

######define a function to generate a block given a first trip
def block_generation(i,block,following_set):
    while True:
        m = return_following(i,following_set)
        if m != -1: 
            block.append(m)
            i = m
        else: 
            print("block=")
            print(block)
            return block

######define a function to get difference of two lists
def diff(list1, list2):
    return (list(list(set(list1)-set(list2)) + list(set(list2)-set(list1))))

######define a function to generate the service trip part of a chromosome
def trippart_chromosome_generation(rest_trips, chromosome, trip_chains, following_set, no_bus, no_trip):
    while True:
        if len(rest_trips) != 0:
            fist_trip = min(rest_trips)                        #get the trip with the earlest starting time in rest of the trips
            new_block = []
            new_block.append(fist_trip)
            new_block = block_generation(fist_trip,new_block,following_set)  #generate a new block
            rest_trips = diff(rest_trips, new_block)           #get rest of the trips
            chromosome = chromosome + new_block                #add the newly generated block to the chromosome
            trip_chains.append(new_block)
            no_bus += 1
            #crosspoint_trip.append(len(chromosome)) 
            #crosspoint_batt.append(no_bus) 
            no_trip.append(len(new_block))
            following_set = {key:[ele for ele in following_set[key] if ele not in chromosome] for key in following_set.keys()}
        else:
            print("All trips have been assigned to blocks")
            print("Generated trip part of chromosome =")
            print(trip_chains)
            #crosspoint_trip.pop()
            #print("Possible crossover points =")
            #print(crosspoint_trip)
            print("Fleet size under this trip schedule =")
            print(no_bus)
            print("Number of trips for each bus =")
            print(no_trip)
            
            return trip_chains, no_bus, no_trip

######define a function to generate the charging station part of the chromosome
def charpart_chromosome_generation():
    Charging_station_part_chromosome = []
    for i in range(cs):
        Charging_station_part_chromosome.append(random.randint(0, 1))
    return Charging_station_part_chromosome

######define a function to randomly assign a battery capacity for each bus in the bus fleet
def battery_cap_generation(fleet_size):
    bus_type = [1,2]                   #available bus type in the market
    battery_type = [90,380]           #battery capacity of each bus type
    battery_size = [0]*fleet_size      #define a list to store the battery size of each bus in the bus fleet
    battery_lower = [0]*fleet_size     #define the lower bound of battery energy in bus
    battery_upper = [0]*fleet_size     #define the upper bound of battery energy in bus
    for i in range(fleet_size):        #randomly assign a battery capacity for each bus in the bus fleet
        battery_size[i] = random.choice(battery_type)
        battery_lower[i] =  battery_size[i]*0.2
        battery_upper[i] = battery_size[i]*0.9
    return battery_size, battery_lower, battery_upper

######define the function to check feasibility of the generated chromosome
def check_feasibility(fleet_size, trips_per_bus, battery_lower, battery_upper, trip_chains):
    i = 0
    while (i < fleet_size):
        print("i=" +str(i))
        a = trips_per_bus[i]                        #get the number of trips for the bus i
        battery_l = battery_lower[i]                #get the battery capacity lower bound of the bus i
        battery_u = battery_upper[i]                #get the battery capacity upper bound of the bus i
        trip_chain = trip_chains[i]                 #get the trip chain of the bus i
        b = trip_chain[0]                           #get the first trip in the trip chain of the bus i
        remain_energy = battery_u - From_depot_to_trip_energy.iloc[b-1][1] - trip_information.iloc[b-1]["Energy_need"]   #get the remaining energy in bus i after finishing its first trip
        if remain_energy < battery_l:
            print("The depot to first trip part of the trip chain "+str(i)+" is not feasible")
            i = 100
            return 0
            break
        else:
            if a == 1:
                remain_energy = remain_energy - From_trip_to_depot_energy.iloc[b-1][1]
                if remain_energy < battery_l:
                    print("The only trip to depot part of the trip chain "+str(i)+" is not feasible")
                    i = 100
                    return 0
                    break
                else:
                    print("Trip chain "+str(i)+" is feasible")
            elif a > 1:
                j = 1
                while (j < a):
                    print("j= "+str(j))
                    bb = trip_chain[j] 
                    print("bb= " +str(bb))
                    cc = trip_chain[j-1] 
                    print("cc= " +str(cc))
                    remain_energy = remain_energy - energy_need.iloc[cc-1][bb]
                    if remain_energy < battery_l:
                        print("The between trips part "+str(cc)+" to "+str(bb)+" of the trip chain "+str(i)+" is not feasible")
                        i = 100
                        return 0
                        break
                    else:
                        if j < a-1:
                            stt = trip_information.iloc[bb-1]["Start_terminal"]     #get the start terminal of the next trip
                            remain_energy = remain_energy + charpart_chromosome[int(stt)-1]*p_bus_max*available_charging_time.iloc[cc-1][bb]
                            if remain_energy > battery_u:
                                remain_energy = battery_u
                            else: 
                                remain_energy = remain_energy
                            remain_energy = remain_energy - trip_information.iloc[bb-1]["Energy_need"]
                            if remain_energy < battery_l:
                                print("The between trips part "+str(bb)+" of the trip chain "+str(i)+" is not feasible")
                                i = 100
                                return 0
                                break
                        else:
                            remain_energy = remain_energy - From_trip_to_depot_energy.iloc[bb-1][1]
                            if remain_energy < battery_l:
                                print("The last trip to depot part of the trip chain "+str(i)+" is not feasible")
                                i = 100
                                return 0
                                break
                            else:
                                print("Trip chain "+str(i)+" is feasible")
                    j += 1
        i += 1
    return 1


######define the maximum charging power of a bus
p_bus_max = 350

### generate the trip part of the chromosome
rest_trips = [i for i in range(117)]
rest_trips.pop(0)
chromosome = []
#crosspoint_trip = []
#crosspoint_batt = []
no_bus = 0
no_trip = []
trip_chains = []
trippart_chromosome = trippart_chromosome_generation(rest_trips, chromosome, trip_chains, following_set, no_bus, no_trip)

#trip_chromosome=trippart_chromosome[0]
trip_chains=trippart_chromosome[0]
#crossover_point=trippart_chromosome[2]
fleet_size=trippart_chromosome[1]
trips_per_bus=trippart_chromosome[2]

### generate the charging station part of the chromosome
charpart_chromosome = charpart_chromosome_generation()
print("Generated charging station part of chromosome =")
print(charpart_chromosome)

### generate the battery capacity for each bus
battery_cap = battery_cap_generation(fleet_size)
battery_size = battery_cap[0]
battery_lower = battery_cap[1]
battery_upper = battery_cap[2]
print(battery_size)
print(battery_lower)
print(battery_upper)

### check feasibility of the generated chromosome
check = check_feasibility(fleet_size, trips_per_bus, battery_lower, battery_upper, trip_chains)

### define a empty dictionary to store feasible chromosomes
dict_feasible_chromosome = {}
ii=41      # number of chromosome    
while (ii<=60):
    if check == 0:
        ### generate the trip part of a new chromosome
        rest_trips = [i for i in range(117)]
        rest_trips.pop(0)
        chromosome = []
        #crosspoint = []
        no_bus = 0
        no_trip = []
        trip_chains = []
        trippart_chromosome = trippart_chromosome_generation(rest_trips, chromosome, trip_chains, following_set, no_bus, no_trip)

        #trip_chromosome=trippart_chromosome[0]
        trip_chains=trippart_chromosome[0]
        #crossover_point=trippart_chromosome[2]
        fleet_size=trippart_chromosome[1]
        trips_per_bus=trippart_chromosome[2]

        ### generate the charging station part of a new chromosome
        charpart_chromosome = charpart_chromosome_generation()
        print("Generated charging station part of chromosome =")
        print(charpart_chromosome)

        ### generate the battery capacity for each bus
        battery_cap = battery_cap_generation(fleet_size)
        battery_size = battery_cap[0]
        battery_lower = battery_cap[1]
        battery_upper = battery_cap[2]
        print(battery_size)
        print(battery_lower)
        print(battery_upper)
    
        ### check feasibility of the generated new chromosome
        check = check_feasibility(fleet_size, trips_per_bus, battery_lower, battery_upper, trip_chains)
    
    else:
        dict_feasible_chromosome[ii] = {"char_chromosome": charpart_chromosome, "fleet_size": fleet_size, "trip_chromosome": trip_chains, "battery_cap": battery_cap}
        print('hello')
        ii += 1
        
        ### generate the trip part of a new chromosome
        rest_trips = [i for i in range(117)]
        rest_trips.pop(0)
        chromosome = []
        #crosspoint = []
        no_bus = 0
        no_trip = []
        trip_chains = []
        trippart_chromosome = trippart_chromosome_generation(rest_trips, chromosome, trip_chains, following_set, no_bus, no_trip)

        #trip_chromosome=trippart_chromosome[0]
        trip_chains=trippart_chromosome[0]
        #crossover_point=trippart_chromosome[2]
        fleet_size=trippart_chromosome[1]
        trips_per_bus=trippart_chromosome[2]

        ### generate the charging station part of a new chromosome
        charpart_chromosome = charpart_chromosome_generation()
        print("Generated charging station part of chromosome =")
        print(charpart_chromosome)

        ### generate the battery capacity for each bus
        battery_cap = battery_cap_generation(fleet_size)
        battery_size = battery_cap[0]
        battery_lower = battery_cap[1]
        battery_upper = battery_cap[2]
        print(battery_size)
        print(battery_lower)
        print(battery_upper)
    
        ### check feasibility of the generated new chromosome
        check = check_feasibility(fleet_size, trips_per_bus, battery_lower, battery_upper, trip_chains)

block=
[1, 4, 11, 17, 22, 31, 38, 42, 51, 56, 63, 68, 73, 78, 86, 90, 99, 105, 111, 116]
block=
[2, 9, 18, 28, 33, 41, 46, 54, 58, 66, 76, 81, 89, 94, 102, 108, 114]
block=
[3, 5, 13, 21, 26, 34, 44, 52, 57, 64, 74, 83, 92, 97, 101, 109, 112, 115]
block=
[6, 7, 16, 25, 32, 43, 48, 59, 70, 79, 88, 96, 107, 113]
block=
[8, 14, 24, 35, 40, 50, 60, 65, 77, 85, 95, 100, 110]
block=
[10, 23, 36, 49, 61, 67, 75, 82, 98, 103]
block=
[12, 19, 30, 37, 53, 62, 72, 84, 104]
block=
[15, 20, 39, 55, 71, 80, 93, 106]
block=
[27, 47, 69, 87]
block=
[29, 91]
block=
[45]
All trips have been assigned to blocks
Generated trip part of chromosome =
[[1, 4, 11, 17, 22, 31, 38, 42, 51, 56, 63, 68, 73, 78, 86, 90, 99, 105, 111, 116], [2, 9, 18, 28, 33, 41, 46, 54, 58, 66, 76, 81, 89, 94, 102, 108, 114], [3, 5, 13, 21, 26, 34, 44, 52, 57, 64, 74, 83, 92, 97, 101, 109, 112, 115], [6, 7, 16, 25, 32, 43, 48, 59, 70, 79, 88, 96, 107, 113], [8, 14, 24, 35, 40, 50, 60, 65, 77, 85, 95, 100, 110], [10, 23, 36, 49, 61, 

block=
[4, 11, 20, 30, 37, 43, 53, 59, 64, 73, 78, 82, 93, 101, 109]
block=
[6, 13, 19, 24, 32, 42, 57, 62, 69, 80, 87, 99, 108]
block=
[8, 16, 31, 36, 47, 58, 74, 92, 103]
block=
[12, 22, 38, 45, 54, 71, 81, 90, 102, 107]
block=
[15, 39, 49, 76, 97]
block=
[21, 26, 52, 63, 86]
block=
[27, 68]
block=
[46]
All trips have been assigned to blocks
Generated trip part of chromosome =
[[1, 5, 9, 14, 23, 28, 35, 40, 48, 55, 60, 65, 70, 75, 83, 88, 95, 100, 104, 110, 113, 116], [2, 7, 17, 25, 33, 41, 50, 61, 67, 72, 79, 85, 91, 96, 105, 111, 114], [3, 10, 18, 29, 34, 44, 51, 56, 66, 77, 84, 89, 94, 98, 106, 112, 115], [4, 11, 20, 30, 37, 43, 53, 59, 64, 73, 78, 82, 93, 101, 109], [6, 13, 19, 24, 32, 42, 57, 62, 69, 80, 87, 99, 108], [8, 16, 31, 36, 47, 58, 74, 92, 103], [12, 22, 38, 45, 54, 71, 81, 90, 102, 107], [15, 39, 49, 76, 97], [21, 26, 52, 63, 86], [27, 68], [46]]
Fleet size under this trip schedule =
11
Number of trips for each bus =
[22, 17, 17, 15, 13, 9, 10, 5, 5, 2, 1]
Generated c

j= 14
bb= 99
cc= 91
j= 15
bb= 107
cc= 99
j= 16
bb= 114
cc= 107
Trip chain 1 is feasible
i=2
j= 1
bb= 7
cc= 3
j= 2
bb= 19
cc= 7
j= 3
bb= 28
cc= 19
j= 4
bb= 34
cc= 28
j= 5
bb= 46
cc= 34
j= 6
bb= 54
cc= 46
j= 7
bb= 59
cc= 54
j= 8
bb= 67
cc= 59
j= 9
bb= 75
cc= 67
j= 10
bb= 84
cc= 75
j= 11
bb= 92
cc= 84
j= 12
bb= 100
cc= 92
j= 13
bb= 108
cc= 100
j= 14
bb= 112
cc= 108
j= 15
bb= 115
cc= 112
Trip chain 2 is feasible
i=3
j= 1
bb= 9
cc= 4
j= 2
bb= 14
cc= 9
The between trips part 14 of the trip chain 3 is not feasible
block=
[1, 2, 9, 16, 24, 32, 40, 47, 55, 62, 67, 74, 82, 90, 100, 107, 110, 114, 116]
block=
[3, 5, 10, 18, 26, 34, 44, 51, 56, 65, 70, 75, 80, 88, 97, 104, 112, 115]
block=
[4, 13, 20, 28, 36, 43, 48, 59, 66, 78, 87, 93, 99, 105, 109, 113]
block=
[6, 12, 22, 31, 41, 49, 58, 68, 76, 81, 91, 102, 106, 111]
block=
[7, 17, 29, 35, 42, 50, 63, 71, 83, 95, 108]
block=
[8, 11, 23, 30, 37, 46, 57, 72, 84, 94, 101]
block=
[14, 25, 39, 53, 61, 73, 85, 89, 103]
block=
[15, 38, 45, 60, 77, 96]

j= 4
bb= 24
cc= 19
The between trips part 24 of the trip chain 0 is not feasible
block=
[1, 5, 11, 18, 27, 34, 42, 50, 60, 67, 72, 81, 89, 94, 102, 106, 110, 115]
block=
[2, 9, 17, 25, 33, 41, 46, 54, 59, 68, 74, 82, 90, 100, 107, 113, 116]
block=
[3, 10, 21, 26, 37, 45, 51, 61, 69, 75, 84, 93, 99, 105, 109, 114]
block=
[4, 13, 22, 35, 43, 53, 58, 66, 77, 86, 96, 111]
block=
[6, 14, 24, 32, 40, 47, 56, 63, 73, 80, 88, 97, 104, 112]
block=
[7, 16, 23, 31, 36, 48, 57, 62, 76, 83, 92, 103, 108]
block=
[8, 19, 30, 38, 49, 64, 71, 78, 95, 101]
block=
[12, 20, 29, 44, 65, 79, 87, 98]
block=
[15, 52, 70, 85, 91]
block=
[28, 55]
block=
[39]
All trips have been assigned to blocks
Generated trip part of chromosome =
[[1, 5, 11, 18, 27, 34, 42, 50, 60, 67, 72, 81, 89, 94, 102, 106, 110, 115], [2, 9, 17, 25, 33, 41, 46, 54, 59, 68, 74, 82, 90, 100, 107, 113, 116], [3, 10, 21, 26, 37, 45, 51, 61, 69, 75, 84, 93, 99, 105, 109, 114], [4, 13, 22, 35, 43, 53, 58, 66, 77, 86, 96, 111], [6, 14, 24, 32, 4

block=
[3, 5, 11, 19, 27, 32, 39, 47, 55, 60, 67, 72, 79, 85, 89, 94, 101, 108, 111, 115]
block=
[4, 10, 18, 29, 34, 45, 49, 54, 61, 66, 74, 82, 91, 100, 105, 113]
block=
[6, 12, 16, 23, 30, 37, 43, 48, 58, 70, 78, 86, 95, 106, 112]
block=
[8, 14, 22, 31, 38, 50, 62, 71, 83, 90, 99, 107]
block=
[9, 24, 36, 41, 53, 63, 68, 77, 84, 92, 102, 109]
block=
[13, 21, 25, 40, 59, 76, 87, 98, 103]
block=
[17, 28, 44, 56, 69, 75, 96]
block=
[33, 51, 80, 93]
block=
[46, 65]
All trips have been assigned to blocks
Generated trip part of chromosome =
[[1, 2, 7, 15, 20, 26, 35, 42, 52, 57, 64, 73, 81, 88, 97, 104, 110, 114, 116], [3, 5, 11, 19, 27, 32, 39, 47, 55, 60, 67, 72, 79, 85, 89, 94, 101, 108, 111, 115], [4, 10, 18, 29, 34, 45, 49, 54, 61, 66, 74, 82, 91, 100, 105, 113], [6, 12, 16, 23, 30, 37, 43, 48, 58, 70, 78, 86, 95, 106, 112], [8, 14, 22, 31, 38, 50, 62, 71, 83, 90, 99, 107], [9, 24, 36, 41, 53, 63, 68, 77, 84, 92, 102, 109], [13, 21, 25, 40, 59, 76, 87, 98, 103], [17, 28, 44, 56, 69, 75

...

block=
[21, 63, 96]
block=
[26, 49, 86]
All trips have been assigned to blocks
Generated trip part of chromosome =
[[1, 4, 10, 19, 27, 32, 39, 47, 52, 57, 64, 73, 80, 89, 94, 101, 108, 112, 116], [2, 8, 14, 22, 31, 38, 42, 53, 60, 65, 70, 74, 82, 91, 100, 107, 110, 114], [3, 5, 9, 17, 25, 34, 45, 51, 56, 67, 72, 81, 88, 95, 105, 111, 115], [6, 11, 18, 28, 36, 41, 46, 54, 61, 66, 76, 84, 93, 97, 104, 113], [7, 15, 23, 33, 44, 50, 59, 71, 78, 87, 99, 106], [12, 20, 29, 35, 48, 55, 62, 77, 83, 92, 103, 109], [13, 24, 40, 58, 69, 79, 85, 90, 98], [16, 30, 37, 43, 68, 75, 102], [21, 63, 96], [26, 49, 86]]
Fleet size under this trip schedule =
10
Number of trips for each bus =
[19, 18, 17, 16, 12, 12, 9, 7, 3, 3]
Generated charging station part of chromosome =
[1, 1, 1, 1]
[380, 380, 380, 380, 380, 380, 90, 380, 90, 380]
[76.0, 76.0, 76.0, 76.0, 76.0, 76.0, 18.0, 76.0, 18.0, 76.0]
[342.0, 342.0, 342.0, 342.0, 342.0, 342.0, 81.0, 342.0, 81.0, 342.0]
i=0
j= 1
bb= 4
cc= 1
j= 2
bb= 10
cc= 4
j= 3

cc= 31
j= 7
bb= 43
cc= 38
j= 8
bb= 51
cc= 43
j= 9
bb= 58
cc= 51
j= 10
bb= 66
cc= 58
j= 11
bb= 75
cc= 66
j= 12
bb= 83
cc= 75
The between trips part 83 of the trip chain 0 is not feasible
block=
[1, 4, 9, 14, 23, 30, 34, 42, 51, 59, 66, 76, 83, 90, 100, 107, 112, 116]
block=
[2, 10, 19, 24, 33, 40, 49, 57, 65, 70, 78, 82, 92, 97, 103, 110, 113]
block=
[3, 7, 16, 25, 35, 44, 53, 58, 68, 75, 85, 89, 98, 105, 109, 115]
block=
[5, 12, 20, 27, 32, 43, 52, 62, 69, 73, 84, 91, 99, 108, 111, 114]
block=
[6, 11, 17, 28, 38, 47, 55, 64, 71, 80, 87, 95, 106]
block=
[8, 21, 31, 36, 41, 50, 63, 72, 79, 88, 101]
block=
[13, 22, 37, 48, 56, 77, 86, 102]
block=
[15, 26, 39, 46, 61, 81, 94, 104]
block=
[18, 54, 93]
block=
[29, 67, 96]
block=
[45, 60, 74]
All trips have been assigned to blocks
Generated trip part of chromosome =
[[1, 4, 9, 14, 23, 30, 34, 42, 51, 59, 66, 76, 83, 90, 100, 107, 112, 116], [2, 10, 19, 24, 33, 40, 49, 57, 65, 70, 78, 82, 92, 97, 103, 110, 113], [3, 7, 16, 25, 35, 44, 53, 58, 

block=
[3, 9, 14, 19, 29, 33, 41, 46, 50, 60, 68, 74, 84, 92, 97, 104, 109, 113]
block=
[5, 11, 18, 30, 38, 45, 53, 61, 67, 72, 79, 86, 96, 105, 111, 114]
block=
[6, 13, 22, 32, 40, 47, 54, 59, 69, 78, 82, 94, 98, 107]
block=
[7, 21, 27, 35, 43, 48, 62, 76, 87, 93, 100, 108]
block=
[12, 25, 49, 65, 70, 80, 91, 102]
block=
[16, 37, 56, 77, 85, 101]
block=
[17, 39, 55, 71, 89]
block=
[23, 28, 63]
All trips have been assigned to blocks
Generated trip part of chromosome =
[[1, 4, 10, 20, 26, 34, 42, 51, 58, 66, 75, 83, 88, 95, 103, 110, 115], [2, 8, 15, 24, 31, 36, 44, 52, 57, 64, 73, 81, 90, 99, 106, 112, 116], [3, 9, 14, 19, 29, 33, 41, 46, 50, 60, 68, 74, 84, 92, 97, 104, 109, 113], [5, 11, 18, 30, 38, 45, 53, 61, 67, 72, 79, 86, 96, 105, 111, 114], [6, 13, 22, 32, 40, 47, 54, 59, 69, 78, 82, 94, 98, 107], [7, 21, 27, 35, 43, 48, 62, 76, 87, 93, 100, 108], [12, 25, 49, 65, 70, 80, 91, 102], [16, 37, 56, 77, 85, 101], [17, 39, 55, 71, 89], [23, 28, 63]]
Fleet size under this trip schedul

j= 1
bb= 5
cc= 1
j= 2
bb= 9
cc= 5
j= 3
bb= 14
cc= 9
j= 4
bb= 22
cc= 14
j= 5
bb= 31
cc= 22
j= 6
bb= 39
cc= 31
j= 7
bb= 46
cc= 39
j= 8
bb= 51
cc= 46
j= 9
bb= 59
cc= 51
j= 10
bb= 64
cc= 59
j= 11
bb= 71
cc= 64
j= 12
bb= 77
cc= 71
j= 13
bb= 81
cc= 77
j= 14
bb= 88
cc= 81
j= 15
bb= 95
cc= 88
j= 16
bb= 103
cc= 95
j= 17
bb= 108
cc= 103
j= 18
bb= 112
cc= 108
j= 19
bb= 116
cc= 112
Trip chain 0 is feasible
i=1
j= 1
bb= 7
cc= 2
j= 2
bb= 15
cc= 7
The between trips part 15 of the trip chain 1 is not feasible
block=
[1, 4, 10, 18, 26, 34, 42, 50, 58, 66, 74, 82, 91, 100, 105, 110, 115]
block=
[2, 9, 16, 23, 31, 38, 43, 51, 56, 63, 68, 73, 78, 86, 94, 101, 108, 111, 116]
block=
[3, 5, 13, 19, 24, 32, 40, 48, 59, 69, 77, 81, 88, 96, 102, 107, 114]
block=
[6, 7, 15, 20, 28, 33, 41, 52, 57, 64, 71, 83, 93, 97, 103, 112]
block=
[8, 11, 22, 35, 45, 54, 61, 65, 75, 84, 89, 99, 106, 113]
block=
[12, 25, 36, 44, 49, 62, 67, 79, 87, 104, 109]
block=
[14, 30, 46, 53, 72, 90, 98]
block=
[17, 29, 39, 60, 76, 85, 9

block=
[2, 10, 20, 26, 34, 42, 53, 59, 64, 71, 80, 90, 99, 105, 109, 112, 115]
block=
[3, 5, 12, 16, 23, 28, 36, 44, 51, 56, 66, 74, 83, 88, 98, 103, 108, 113]
block=
[6, 11, 19, 24, 32, 41, 48, 55, 63, 70, 78, 82, 91, 100, 111]
block=
[7, 17, 25, 37, 47, 57, 67, 77, 86, 96, 102, 106]
block=
[8, 18, 29, 38, 45, 52, 61, 73, 87, 95, 104]
block=
[13, 21, 33, 46, 62, 69, 81, 93]
block=
[15, 30, 49, 58, 76, 92]
block=
[31, 40, 68, 94]
block=
[39, 84]
block=
[54, 75]
All trips have been assigned to blocks
Generated trip part of chromosome =
[[1, 4, 9, 14, 22, 27, 35, 43, 50, 60, 65, 72, 79, 85, 89, 97, 101, 107, 110, 114, 116], [2, 10, 20, 26, 34, 42, 53, 59, 64, 71, 80, 90, 99, 105, 109, 112, 115], [3, 5, 12, 16, 23, 28, 36, 44, 51, 56, 66, 74, 83, 88, 98, 103, 108, 113], [6, 11, 19, 24, 32, 41, 48, 55, 63, 70, 78, 82, 91, 100, 111], [7, 17, 25, 37, 47, 57, 67, 77, 86, 96, 102, 106], [8, 18, 29, 38, 45, 52, 61, 73, 87, 95, 104], [13, 21, 33, 46, 62, 69, 81, 93], [15, 30, 49, 58, 76, 92], [3

In [9]:
dict_feasible_chromosome

{41: {'char_chromosome': [1, 1, 1, 1],
  'fleet_size': 10,
  'trip_chromosome': [[1,
    4,
    10,
    20,
    27,
    32,
    39,
    44,
    51,
    56,
    63,
    71,
    78,
    82,
    90,
    100,
    107,
    113,
    116],
   [2, 8, 15, 24, 33, 41, 49, 54, 59, 66, 75, 83, 88, 95, 101, 108, 112, 115],
   [3, 9, 14, 22, 35, 40, 50, 61, 68, 74, 85, 92, 97, 105, 111, 114],
   [5, 12, 16, 26, 36, 45, 55, 60, 69, 77, 86, 91, 96, 104, 110],
   [6, 13, 18, 29, 37, 43, 48, 58, 67, 76, 87, 94, 103, 109],
   [7, 19, 30, 38, 47, 57, 62, 72, 81, 89, 98, 106],
   [11, 17, 28, 42, 53, 65, 73, 84, 99],
   [21, 25, 46, 70, 93, 102],
   [23, 34, 79],
   [31, 52, 64, 80]],
  'battery_cap': ([90, 380, 380, 380, 380, 380, 90, 380, 380, 90],
   [18.0, 76.0, 76.0, 76.0, 76.0, 76.0, 18.0, 76.0, 76.0, 18.0],
   [81.0, 342.0, 342.0, 342.0, 342.0, 342.0, 81.0, 342.0, 342.0, 81.0])},
 42: {'char_chromosome': [1, 1, 1, 0],
  'fleet_size': 11,
  'trip_chromosome': [[1,
    5,
    10,
    18,
    26,
    3

In [10]:
import pickle
with open('Feasible_chromosome_3.pickle', 'wb') as handle:
    pickle.dump(dict_feasible_chromosome, handle, protocol=pickle.HIGHEST_PROTOCOL)